* Zero-Pi circuit space
* Distribution of degeneracy and insensitivity
* Fixing Josephson elements : EcJ == EJ := E0
* 2-D parameter space : Ec/E0 & El/E0

In [1]:
from DiSuQ.Torch import models
from torch import tensor,stack
from DiSuQ import utils
from numpy import arange,linspace,meshgrid,array,log,argsort,pi,logspace
from DiSuQ.utils import plotCompare,plotHeatmap,plotBox,plotOptimization
from DiSuQ.Torch.optimization import uniformParameters,truncNormalParameters,initializationSequential
from DiSuQ.Torch import optimization
from DiSuQ.Torch.optimization import lossDegeneracyTarget,lossDegeneracyWeighted
from DiSuQ.Torch.components import indE,capE
from torch import set_num_threads
set_num_threads(32)

In [2]:
L_ = indE(1e-7); L0 = indE(5*1e-9); print('Inductance Bound(GHz):',L_,L0+L_)
C0 = capE(5*1e-16); C_ = capE(1e-14*5) ; print('Capacitance Bound(GHz):',C_,C0+C_)
CJ0 = capE(5*1e-16/4); CJ_ = capE(1e-14/2) ; print('Shunt Bound(GHz):',CJ_,CJ0+CJ_)
J0 = 25. ; J_ = 5. ; print('Junction Bound(GHz):',J_,J0+J_)

Inductance Bound(GHz): 1.6346151294983178 34.32691771946467
Capacitance Bound(GHz): 0.38740458615415185 39.12786320156934
Shunt Bound(GHz): 3.8740458615415188 158.83588032320228
Junction Bound(GHz): 5.0 30.0


In [3]:
pairs = {'Lx':'Ly','Cx':'Cy','Jx':'Jy','CJx':'CJy'}
E0 = 30.

In [4]:
lower,upper = -2.5,0 ; n_scape = 100
El = logspace(lower,upper,n_scape,endpoint=True)
Ec = logspace(lower,upper,n_scape,endpoint=True)
scape = {'Lx':El*E0,'Cx':Ec*E0}

## Charge Basis

In [5]:
basis = [8,8,8] ; rep = 'Q'
flux_point = ['Lx','Ly']
circuit = models.zeroPi(basis,Ej=E0,Ec=5.,El=10.,EcJ=E0,sparse=False,symmetry=True,
                        _L_=(L_,L0),_C_=(C_,C0),_J_=(J_,J0),_CJ_=(CJ_,CJ0))
static = circuit.circuitState()

In [6]:
flux_profile = [{'Lx':tensor(.5),'Ly':tensor(0.0)},{'Lx':tensor(.45),'Ly':tensor(0.0)}]

In [7]:
optimizer = optimization.OrderingOptimization(circuit,representation=rep)

### Degeneracy

In [8]:
D0 = 1; delta0 = 0
lossFunction = lossDegeneracyWeighted(delta0,D0)

In [9]:
LossD = optimizer.lossScape(lossFunction,flux_profile,scape,static)

In [21]:
utils.plotOptimization(LossD.transpose(),El,Ec,dict(),'Degeneracy distribution','El/E0','Ec/E0',export='pdf',
                       size=(500,800),legend=False,log=True)

### Delta

In [11]:
D0 = 0; delta0 = 1
lossFunction = lossDegeneracyWeighted(delta0,D0)

In [ ]:
LossDelta = optimizer.lossScape(lossFunction,flux_profile,scape,static)

In [20]:
utils.plotOptimization(LossDelta.transpose(),El,Ec,dict(),'Delta distribution','El/E0','Ec/E0',export='pdf',
                 size=(500,800),legend=False,log=True)

### Flux Operator validation
* Maximum potential energy $U^*$ representable is bound by $\Phi_{max}$
* Validation
    - $E_{0,1,2} < U^*$
    - $\langle n |\Phi| n \rangle$, for $n=0,1,2,3$

## Fourier transform

* higher fluctuation fails numerical differentiation
* higher density of local minimas
* dominant higher frequencies

## Kerman Basis

In [27]:
basis = {'O':[10,10],'I':[],'J':[10]} ; rep = 'K'
flux_point = ['Lx','Ly']
circuit = models.zeroPi(basis,Ej=E0,Ec=5.,El=10.,EcJ=E0,sparse=False,symmetry=True,
                        _L_=(L_,L0),_C_=(C_,C0),_J_=(J_,J0),_CJ_=(CJ_,CJ0))
static = circuit.circuitState()

In [28]:
circuit.modeDistribution()

(2, 0, 1)

In [29]:
flux_profile = [{'Lx':tensor(.5),'Ly':tensor(0.0)},{'Lx':tensor(.45),'Ly':tensor(0.0)}]

In [30]:
optimizer = optimization.OrderingOptimization(circuit,representation=rep)

### Degeneracy

In [31]:
D0 = 1; delta0 = 0
lossFunction = lossDegeneracyWeighted(delta0,D0)

In [ ]:
LossD = optimizer.lossScape(lossFunction,flux_profile,scape,static)

In [ ]:
utils.plotOptimization(LossD.transpose(),El,Ec,dict(),'Degeneracy Kerman-Basis','El/E0','Ec/E0',export='pdf',
                       size=(600,800),legend=False,log=True)

In [37]:
utils.plotOptimization(LossD.transpose(),El,Ec,dict(),'Degeneracy Kerman-Basis','El/E0','Ec/E0',export='pdf',
                       size=(600,800),legend=False,log=False)

### Delta

In [ ]:
D0 = 0; delta0 = 1
lossFunction = lossDegeneracyWeighted(delta0,D0)

In [ ]:
LossDelta = optimizer.lossScape(lossFunction,flux_profile,scape,static)

In [ ]:
utils.plotOptimization(LossDelta.transpose(),El,Ec,dict(),'Delta Kerman-Basis','El/E0','Ec/E0',export='pdf',
                 size=(600,800),legend=False,log=True)

In [38]:
utils.plotOptimization(LossDelta.transpose(),El,Ec,dict(),'Delta Kerman-Basis','El/E0','Ec/E0',export='pdf',
                 size=(600,800),legend=False,log=False)

## Oscillator Basis

In [5]:
basis = [20,20,20] ; rep = 'O'
flux_point = ['Lx','Ly']
circuit = models.zeroPi(basis,Ej=E0,Ec=5.,El=10.,EcJ=E0,sparse=False,symmetry=True,
                        _L_=(L_,L0),_C_=(C_,C0),_J_=(J_,J0),_CJ_=(CJ_,CJ0))
static = circuit.circuitState()

In [6]:
circuit.modeDistribution()

(2, 0, 1)

In [7]:
flux_profile = [{'Lx':tensor(.5),'Ly':tensor(0.0)},{'Lx':tensor(.45),'Ly':tensor(0.0)}]

In [8]:
optimizer = optimization.OrderingOptimization(circuit,representation=rep)

### Degeneracy

In [9]:
D0 = 1; delta0 = 0
lossFunction = lossDegeneracyWeighted(delta0,D0)

In [ ]:
LossD = optimizer.lossScape(lossFunction,flux_profile,scape,static)

In [ ]:
utils.plotOptimization(LossD,El,Ec,dict(),'Degeneracy Oscillator-Basis','El/E0','Ec/E0',export='pdf',
                       size=(600,800),legend=False,log=False)

### Delta

In [ ]:
D0 = 0; delta0 = 1
lossFunction = lossDegeneracyWeighted(delta0,D0)

In [ ]:
LossDelta = optimizer.lossScape(lossFunction,flux_profile,scape,static)

In [ ]:
utils.plotOptimization(LossDelta,El,Ec,dict(),'Delta Oscillator-Basis','El/E0','Ec/E0',export='pdf',
                 size=(600,800),legend=False,log=False)

### Flux Operator validation
* Maximum potential energy $U^*$ representable is bound by $\Phi_{max}$
* Validation
    - $E_{0,1,2} < U^*$
    - $\langle n |\Phi| n \rangle$, for $n=0,1,2,3$

## Fourier transform

* higher fluctuation fails numerical differentiation
* higher density of local minimas
* dominant higher frequencies